#                       FELIPE BASURTO - 15/12/2020
##                    https://github.com/felipebasurto

In [3]:
# -IMPORTACIONES- 


import numpy as np  
import os
from libsvm import svmutil
import sys
from skimage.io import imread
from libsvm.svmutil import *
from sklearn.model_selection import cross_val_predict, train_test_split
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from skimage.transform import resize
from sklearn.neural_network import MLPClassifier
import warnings
import pandas as pd

warnings.filterwarnings("ignore")

In [4]:
# - CARGAMOS LA BASE DE DATOS -

excel_file = 'BBDD.xlsx'
df = pd.read_excel(excel_file)

# Genero otra "matriz" con todas las filas, pero sin las columnas de: (1) "Número de agujeros", y (2) "Etiquetas/Clases".
x = df.drop(['Número de agujeros','Etiqueta/Clase'],axis=1)
x = x.values
#Ésto nos quita la primera columna, la cual nos es inservible
x = x[:, 1:]

# Como la columna de etiquetas contiene tanto números como letras, hay que transformar esta columna a una representación numérica 
# única, para que pueda ser usada por el algoritmo SVM.
from sklearn.preprocessing import LabelEncoder
y = df.values
y = y[0:,258].astype(str)

#Convertimos el array Y en valores numéricos
le = LabelEncoder()
y = le.fit_transform(y)

In [5]:
# - FUNCIÓN EVALÚA -

def evalua(Y_test, Y_pred):
    aciertos = sum(t == p for t, p in zip(Y_test, Y_pred))
    tam = len(Y_pred)
    return 'Accuracy = {:.6}% ({}/{})'.format(100 * aciertos/tam, aciertos, tam)

In [6]:
print('-------------------------------------------------------')
print('Cantidad de matrices / tamaño de cada una =',x.shape)
print('Cantidad de etiquetas =',y.shape)
print('-------------------------------------------------------')

-------------------------------------------------------
Cantidad de matrices / tamaño de cada una = (2068, 256)
Cantidad de etiquetas = (2068,)
-------------------------------------------------------


In [7]:
print('-------------------------------------------------------')
print('PERCEPTRÓN MULTICAPA (SkLearn)')
print('-------------------------------------------------------')
X_train, X_test, Y_train, Y_test = train_test_split(x, y, stratify=y, train_size = 0.7)

classifier = MLPClassifier(random_state=1)
classifier.fit(X_train, Y_train)
Y_pred = classifier.predict(X_test)
evalua(Y_test, Y_pred)

-------------------------------------------------------
PERCEPTRÓN MULTICAPA (SkLearn)
-------------------------------------------------------


'Accuracy = 86.7955% (539/621)'

In [8]:
print('-------------------------------------------------------')
print('SVM (Kernel lineal)')
print('-------------------------------------------------------')

X_train_svm, X_test_svm, y_train_svm, y_test_svm = train_test_split(x, y, stratify = y, train_size = 0.7)

problema = svm_problem(y_train_svm, X_train_svm)
param = svm_parameter('-s 0 -t 0 -b 1 -c 1')
modelo_svm = svm_train(problema, param)
p_label, p_acc, p_val = svm_predict(y_test_svm, X_test_svm, modelo_svm)

print('\n\t\t..................\n')

print('-------------------------------------------------------')
print('SVM (Kernel polinómico)')
print('-------------------------------------------------------')
problema = svm_problem(y_train_svm, X_train_svm)
param = svm_parameter('-s 0 -t 1 -b 1 -c 1')
modelo_svm = svm_train(problema, param)

p_label, p_acc, p_val = svm_predict(y_test_svm, X_test_svm, modelo_svm)

-------------------------------------------------------
SVM (Kernel lineal)
-------------------------------------------------------
Model supports probability estimates, but disabled in predicton.
Accuracy = 98.3897% (611/621) (classification)

		..................

-------------------------------------------------------
SVM (Kernel polinómico)
-------------------------------------------------------
Model supports probability estimates, but disabled in predicton.
Accuracy = 99.3559% (617/621) (classification)
